# Import libraries

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from google.colab import files
import io
import pandas as pd
from difflib import Differ
import string


ModuleNotFoundError: No module named 'google'

#Import data

In [ ]:
# Après exécution de cette cellule, vous devez charger le jeu de données précédemment téléchargé
uploaded = files.upload()

In [ ]:
df = pd.read_json(io.BytesIO(uploaded['train_l.jsonl']), lines=True)
df

#Regroup pairs

In [ ]:
df["GroupID"] = df["qID"].str[:-2]
df

#1. Special case / alternate

In [ ]:
# Fonction de nettoyage des mots (minuscules et suppression de la ponctuation)
def clean_word(word):
    # Convertir en minuscules
    word = word.lower()
    # Supprimer la ponctuation
    word = word.translate(str.maketrans('', '', string.punctuation))
    return word

In [ ]:
def find_differences(group):
    if len(group) == 2:  # Si le groupe a exactement 2 phrases
        sentence1, sentence2 = group.iloc[0]["sentence"], group.iloc[1]["sentence"]

        words1 = set(sentence1.split())  # Découpe la phrase en mots
        words2 = set(sentence2.split())  # Découpe l'autre phrase en mots

        diff1 = words1 - words2  # Mots dans phrase1 mais pas dans phrase2
        diff2 = words2 - words1  # Mots dans phrase2 mais pas dans phrase1

        # Nettoyer les différences
        cleaned_diff1 = {clean_word(word) for word in diff1}
        cleaned_diff2 = {clean_word(word) for word in diff2}

        # Retourner les différences sous forme de chaînes de caractères distinctes
        return pd.Series([", ".join(cleaned_diff1), ", ".join(cleaned_diff2)], index=['diff1', 'diff2'])

    elif len(group) == 1:  # Si une seule phrase dans le groupe
        return pd.Series([None, None], index=['diff1', 'diff2'])

    return pd.Series([None, None], index=['diff1', 'diff2'])

# Appliquer la fonction aux groupes
df_diff = df.groupby("GroupID").apply(find_differences).reset_index()

# Étape 3 : Ajouter les différences dans le DataFrame original
df = pd.merge(df, df_diff, on="GroupID", how="left")
df


In [ ]:
df['sorted_pair'] = df.apply(lambda row: tuple(sorted([row['diff1'], row['diff2']])), axis=1)

# Utiliser groupby pour compter les occurrences des paires
pair_counts = df.groupby('sorted_pair').size().reset_index(name='count')
pair_counts['count'] = pair_counts['count'] / 2

# Trier pour afficher le classement
ranked_pairs = pair_counts.sort_values(by='count', ascending=False).reset_index(drop=True)

print(ranked_pairs)

In [ ]:
# Convertir les tuples de sorted_pair en chaînes de caractères
ranked_pairs['sorted_pair_str'] = ranked_pairs['sorted_pair'].apply(lambda x: ', '.join(map(str, x)))

# Réinitialiser l'index pour éviter les problèmes avec MultiIndex
ranked_pairs = ranked_pairs.reset_index(drop=True)

# Créer un graphique à barres des paires les plus fréquentes
plt.figure(figsize=(12, 8))

# Tracer le graphique avec seaborn
sns.barplot(x='count', y='sorted_pair_str', data=ranked_pairs.head(15), palette='viridis')

# Ajouter les titres et les labels
plt.title('Top 15 des Paires les Plus Fréquentes')
plt.xlabel('Nombre d\'Occurrences')
plt.ylabel('Paires Triées')

# Afficher le graphique
plt.show()

# 2. Name

In [ ]:
def classify_word(word):
    # Vérifier si le mot est un nom propre ou un nom commun
    if word[0].isupper():
        return "proper name"
    else:
        return "common name"

# Appliquer la fonction aux colonnes option1 et option2
df['option1_type'] = df['option1'].apply(classify_word)
df['option2_type'] = df['option2'].apply(classify_word)

# Ajouter une colonne qui combine les deux informations
df['options_type'] = df.apply(
    lambda row: f"{row['option1_type']} / {row['option2_type']}", axis=1
)

df

In [ ]:
sns.countplot(y='options_type', data=df, order=df['options_type'].value_counts().index, palette='viridis')